In [1]:
import pandas as pd
import seaborn as sns
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.max_columns', 50)
%matplotlib inline

In [2]:
df = pd.read_csv('../evaluation/2017_human_evaluation/all_data_final_scores_anonymised.csv',
                usecols=['X_unit_id',
                         'X_worker_id',
                         'fluency',
                         'grammaticality',
                         'semantic_adequacy',
                         'mr',
                         'team',
                         'text',
                         'category',
                         'type',
                         'bleu',
                         'meteor',
                         'ter'
                        ])

df.head()

,X_unit_id,X_worker_id,fluency,grammaticality,semantic_adequacy,mr,team,text,category,type,bleu,meteor,ter
0,1433935686,13581319,3,1,1,"Elliot_See | deathDate | ""1966-02-28""",tilburg-smt,"on february 28, 1966.",Astronaut,seen,0.074,0.354628,75.0
1,1433935686,43942797,3,3,3,"Elliot_See | deathDate | ""1966-02-28""",tilburg-smt,"on february 28, 1966.",Astronaut,seen,0.074,0.354628,75.0
2,1433935687,43891892,3,2,3,Batagor | dishVariation | Shumai,pkuwriter,shumai can be included in arrabbiaita sauce.,Food,seen,0.199,0.161864,75.0
3,1433935687,43942797,3,3,3,Batagor | dishVariation | Shumai,pkuwriter,shumai can be included in arrabbiaita sauce.,Food,seen,0.199,0.161864,75.0
4,1433935687,44014780,1,1,1,Batagor | dishVariation | Shumai,pkuwriter,shumai can be included in arrabbiaita sauce.,Food,seen,0.199,0.161864,75.0


In [88]:
df['weird_eval'] = (((df.semantic_adequacy == 1) | (df.grammaticality == 1) | (df.fluency == 1)) &\
                    ((df.meteor >= 0.8) | (df.bleu >= 0.8) | (df.ter < 20)))

df.weird_eval.value_counts()

False    6014
True       97
Name: weird_eval, dtype: int64

In [89]:
df_workers = df.groupby('X_worker_id').weird_eval.value_counts().unstack(fill_value=0)

df_workers.nlargest(5, True)

weird_eval,False,True
X_worker_id,,
6352345,20,8
21875134,122,4
43856561,110,4
43959305,38,4
12115558,82,3


In [90]:
df_workers = df.groupby(['X_worker_id', 
                         'fluency', 
                         'semantic_adequacy', 
                         'grammaticality']).size().unstack(level=[-1, -2, -3], fill_value=0)

df_workers.head()

grammaticality     1  3      2  1  2  1  2  3  1 ...      2  3     1        2  \
semantic_adequacy  1  1   3  1  2  2  3  3  3  1 ...  3   3  1     2  3  1  1   
fluency            1  3   3  1  1  1  1  1  1  2 ...  3   3  2  1  2  2  3  3   
X_worker_id                                      ...                            
10959404           1  1   6  0  0  0  0  0  0  0 ...  0   0  0  0  0  0  0  0   
11131207           9  5  59  2  3  2  1  2  1  1 ...  3  10  0  0  0  0  0  0   
11257883           0  0   5  0  0  0  0  0  0  0 ...  0   1  0  0  0  0  0  0   
11460728           0  0   3  0  0  0  0  3  2  0 ...  0   0  1  0  0  0  0  0   
11521789           0  0   0  0  0  0  0  0  0  0 ...  0   0  0  1  0  0  0  0   

grammaticality     1  3  
semantic_adequacy  2  2  
fluency            3  1  
X_worker_id              
10959404           0  0  
11131207           0  0  
11257883           0  0  
11460728           0  0  
11521789           0  0  

[5 rows x 27 columns]

In [92]:
patterns_ammount = (df_workers != 0).sum(axis=1)
total_scores = df_workers.sum(axis=1)
n_weird = df[df.weird_eval].X_worker_id.value_counts()

workers_stats = pd.concat([patterns_ammount, total_scores, n_weird], axis=1, keys=['n_patterns', 'n_scores', 'n_weird'])
workers_stats.fillna(0, inplace=True)
workers_stats.nlargest(10, ['n_weird', 'n_scores'])

/home/abevieiramota/anaconda3/envs/webnlg/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """


,n_patterns,n_scores,n_weird
21875134,20,126,4.0
23126666,23,117,3.0
43856561,13,114,4.0
12115558,18,85,3.0
44019926,15,66,3.0
38182446,12,64,3.0
43959305,14,42,4.0
17950689,12,30,3.0
44067725,16,30,3.0
6352345,7,28,8.0


In [95]:
df[df.weird_eval][['mr', 'text', 'meteor', 'fluency', 'grammaticality', 'semantic_adequacy']]

,mr,text,meteor,fluency,grammaticality,semantic_adequacy
35,Denmark | demonym | Danes,denmark demonym is danes.,1.000000,1,1,2
36,Denmark | demonym | Danes,denmark demonym is danes.,1.000000,3,1,3
115,Buzz_Aldrin | was selected by NASA | 1963,buzz aldrin was selected by nasa in 1963.,1.000000,3,1,3
134,"Ben_Urich | fullName | ""Benjamin Urich""",ben urich ' s full name is actually benjamin urich .\n,1.000000,3,1,3
143,California | gemstone | Benitoite,the gemstone of california is benitoite .\n,1.000000,3,1,3
171,Azerbaijan | leader | Artur_Rasizade,artur rasizade is the leader of azerbaijan.,1.000000,3,1,3
192,"Asher_and_Mary_Isabelle_Richardson_House | ReferenceNumber in the National Register of Historic Places | ""88002539""",asher and mary isabelle richardson house has the reference number 88002539 in the national register of historic places.,1.000000,3,1,3
198,Aenir | language | English_language,aenir is written in english.,1.000000,3,1,3
199,Aenir | language | English_language,aenir is written in english.,1.000000,3,3,1
200,Aenir | language | English_language,aenir is written in english.,1.000000,3,2,1


In [101]:
df.groupby(['X_unit_id', 'fluency', 'semantic_adequacy', 'grammaticality']).size().unstack(level=[-1, -2, -3], fill_value=0)

grammaticality     1  3    1  2  3  2     1  2  3        1  2  3  1     2     \
semantic_adequacy  1  3    1  3  2  2  3  3  1  3  1  2  2  2  3  3  2  2  3   
fluency            3  3    1  3  3  3  2  3  2  2  3  2  1  2  1  1  2  1  1   
X_unit_id                                                                      
1433935686         1  1    0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   
1433935687         0  1    1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   
1433935688         0  1    1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   
1433935689         0  3    0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0   
1433935690         0  3    0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0   
1433935691         0  3    0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0   
1433935692         0  3    0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0   
1433935693         0  1    2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   
1433935694         0  0    0  0  0  0  0  0  1  1  1  0  0  0  0  0  0  0  0   
1433935695         0  1    1  0  1  0  0  0  0  0  0  1  0  0  0  0  0  0  0   
1433935696         0  1    0  0  0  1  0  1  0  0  0  0  1  0  0  0  0  0  0   
1433935697         0  3    0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0   
1433935698         0  1    0  0  0  0  0  0  1  1  1  0  0  0  0  0  0  0  0   
1433935699         0  1    0  1  0  0  1  0  0  0  0  0  0  1  0  0  0  0  0   
1433935700         0  1    0  0  0  0  0  0  0  1  0  0  0  1  1  0  0  0  0   
1433935701         0  3    0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0   
1433935702         0  3    0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0   
1433935703         0  3    0  0  0  0  1  0  0  0  0  0  0  1  0  0  0  0  0   
1433935704         0  4    0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0   
1433935705         0  1    0  0  0  0  0  0  2  0  0  0  0  2  0  0  0  0  0   
1433935706         0  3    0  0  0  0  1  0  0  0  0  0  0  1  0  0  0  0  0   
1433935707         0  2    0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0   
1433935708         0  0    0  0  0  0  0  0  0  1  0  0  0  2  0  0  0  0  0   
1433935709         0  0    1  0  0  0  0  0  0  0  0  0  0  0  0  1  1  0  0   
1433935710         0  0    0  1  0  0  2  0  0  0  0  0  0  0  0  0  0  1  1   
1433935711         1  2    0  1  1  0  0  0  0  0  0  0  0  1  0  0  0  0  0   
1433935712         0  4    0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   
1433935713         0  1    1  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0   
1433935714         0  2    0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0   
1433935715         0  3    0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0   
...               .. ..  ... .. .. .. .. .. .. .. .. .. .. .. .. .. .. .. ..   
1461482206         0  0    1  0  0  0  0  0  1  0  0  1  0  0  0  0  0  0  0   
1461482207         0  0    1  0  0  0  0  0  0  0  0  0  1  0  1  0  0  0  0   
1461482208         0  0    1  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0   
1461482209         0  0    3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   
1461482210         0  0    0  1  0  0  1  0  0  0  0  0  1  0  0  0  0  0  0   
1461482211         0  0    3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   
1461482212         0  0    1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   
1461482213         0  0    0  0  0  0  0  0  0  0  0  0  2  0  0  0  1  0  0   
1461482214         0  0    2  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0   
1461482215         0  0    2  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0   
1461482216         0  0    3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   
1461482217         0  0    0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0   
1461482218         0  0    1  0  0  0  0  0  0  0  0  0  0  0  0  1  1  0  0   
1461482219         0  0    1  0  0  0  1  0  0  1  0  0  0  0  0  0  0  0  0   
1461482220         0  0    0  0  0  0  0  0  0  0  0  0  0  0  0  3  0  0  0   


In [102]:
df[df.X_unit_id == '1433935687']

,X_unit_id,X_worker_id,fluency,grammaticality,semantic_adequacy,mr,team,text,category,type,bleu,meteor,ter,weird_eval
2,1433935687,43891892,3,2,3,Batagor | dishVariation | Shumai,pkuwriter,shumai can be included in arrabbiaita sauce.,Food,seen,0.199,0.161864,75.0,False
3,1433935687,43942797,3,3,3,Batagor | dishVariation | Shumai,pkuwriter,shumai can be included in arrabbiaita sauce.,Food,seen,0.199,0.161864,75.0,False
4,1433935687,44014780,1,1,1,Batagor | dishVariation | Shumai,pkuwriter,shumai can be included in arrabbiaita sauce.,Food,seen,0.199,0.161864,75.0,False


In [107]:
df[df.X_worker_id == '43942797'][['mr', 'text', 'meteor', 'fluency', 'grammaticality', 'semantic_adequacy']]

,mr,text,meteor,fluency,grammaticality,semantic_adequacy
1,"Elliot_See | deathDate | ""1966-02-28""","on february 28, 1966.",0.354628,3,3,3
3,Batagor | dishVariation | Shumai,shumai can be included in arrabbiaita sauce.,0.161864,3,3,3
6,"1001_Gaussia | formerName | ""1923 OAA907 XC""",the operator of antwerp is mayor.,0.064257,3,3,3
119,California | gemstone | Benitoite,the fossil is a fossil from california.,0.266409,1,2,1
124,United_States_Air_Force | battles | 1986_United_States_bombing_of_Libya,the united states air force was involved in battles at the 1986 united states bombing of libya.,0.455415,3,3,3
129,"Abradab | background | ""solo_singer""",the administrative government is governed by the flemish government.,0.050477,3,3,3
135,"Ben_Urich | fullName | ""Benjamin Urich""",ben urich ' s full name is actually benjamin urich .\n,1.000000,2,2,2
140,United_States_Air_Force | battles | 1986_United_States_bombing_of_Libya,united states air force fought battles in 1986 united states bombing of libya.,0.422843,3,3,3
144,California | gemstone | Benitoite,the gemstone of california is benitoite .\n,1.000000,3,3,3
148,"Kerala | has to its northwest | Mahé,_India","mahe, india is to the northwest of kerala.",0.477901,3,3,3


In [6]:
df_ = df.groupby(['X_unit_id', 'team'])[['fluency', 'grammaticality', 'semantic_adequacy']].mean()
df_.groupby('team').mean().sort_values('fluency', ascending=False)

,fluency,grammaticality,semantic_adequacy
team,,,
webnlg,2.584753,2.614574,2.771001
vietnam,2.498810,2.536012,2.659226
upf-forge,2.343274,2.469208,2.677877
pkuwriter,2.336024,2.385202,2.075037
melbourne,2.269866,2.394494,2.291369
adapt,2.259449,2.315848,2.298661
baseline,2.248729,2.361136,1.858146
tilburg-pipe,2.197069,2.332496,2.348241
tilburg-nmt,2.012128,2.157961,1.984301


In [5]:
df.groupby('team')[['fluency', 'grammaticality', 'semantic_adequacy']].mean().sort_values('fluency', ascending=False)

,fluency,grammaticality,semantic_adequacy
team,,,
webnlg,2.589905,2.611987,2.772871
upf-forge,2.360555,2.477658,2.681048
pkuwriter,2.331279,2.375963,2.073960
melbourne,2.278125,2.392187,2.312500
adapt,2.269817,2.324695,2.307927
baseline,2.260062,2.366873,1.852941
tilburg-pipe,2.156303,2.280672,2.302521
tilburg-nmt,2.031201,2.177847,2.001560
tilburg-smt,1.821321,1.965465,2.414414


In [113]:
df.team.value_counts()

tilburg-smt     666
adapt           656
upf-forge       649
pkuwriter       649
baseline        646
tilburg-nmt     641
melbourne       640
webnlg          634
tilburg-pipe    595
vietnam         335
Name: team, dtype: int64